Here, we get relevant data and parse a html converted from a pdf found at the url: of the top chinese characters:   

http://www.zein.se/patrick/3000char.html

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
# Suppress just SettingWithCopyWarningda
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

### get initial data of simpilifed and traditional characters, along with phrases/multi-character words in traditional

Loop through the tables of the html. With the exception of the first one, they follow a predictable pattern of definition, words from by multiple characters. 

We create an initial datasets: one with characters only (both simplifed and traditional), together another with multiple characters and their pronunciations. The character category(cat) is the index of the character (used for ranking relatively difficulty).

For alternative characters (mainly numbers) we only show it for the single character word

Special care is used to ensure merged traditional forms can be separated based on the word and meaning, such as (幹乾=>干)

In [2]:
#our key helper function to parse definitions and pronunciations apart given the raw main character form, etc
#returns a list of lists with the order ["cat", "word/charcter", "pronunciation", "definition", "code","alt"]. Code is if the word uses traditional/simplified (s,t,a). More contains extra information (in few cases where there is an alternative character)
# since many words may have compound simplified characters in this list, we will later need to convert again for the words that 
#note pronunciation is the the pronunciation of the character in the word
# are desired to only be written in traditional.
#Therefore, each row will be written twice, one for traditional one for simplified
#for single characters, we use the first pronunciation listed

def getbetween(input, start, end): #helper function to get the string in between two symbols (the first matching abtter)
    startind = input.find(start)
    #if the first one is not found, return empty:
    if startind == -1: 
        return ""
    endind = input.find(end)
    #if the first one is not found (but start is found), return the string starting from the end 
    if endind == -1:
        return input[startind+1:]
    return input[startind+len(start):endind].strip() #otherwise, return 

#current helper function to get all text between characters (a list). Add space to the list, to not include whitespace
def getall_betweenchars(charlist, text):
    #if the length of the list is empty
    if len(charlist) == 0:
        return "" #empty string
    if len(charlist) == 1: #if there is only element, return the only one
        return charlist[0] #empty string
    #get the max and lowest indices 
    charlist.append(" ")
    max = 0 
    max_string = "" #string with the min index
    min = 1000000 #absurdly large
    min_string = "" 
    for char in charlist:
        #find the index of the charcter in the, text
        ind = text.find(char)
        if ind > max:
            max = ind
            max_string = char
        if ind < min:
            min = ind
            min_string = char
    l2 = len(max_string)
    return text[min:max+l2]

def get_wordList(main, des, cat):
    ret = [] #list to return

    simp = main.split("(")[0] #get the simplified form
    alt = getbetween(main,"(A",")") #set as empty string initially for the alternative form #check for alternative form, by default if there is non, this function should return an empty string 
    #the the traditional form, if any 
    trad = getbetween(main,"(F",")")
    if trad == "": #if function is empty, get to be the same as the simplified character
        trad = simp
    print("current simp. :"+simp+", current trad. :"+trad)
    frags = des.split("[") #split on [
    #iterate through diffenent fragments with pronunciations and descriptions within the definition
    #count the number of total definitions
    i = 0
    for frag in frags:
        #get the separate pronunciations
        pro = frag.split("]")[0]
        print("current pronunciation:"+pro)
        #if not empty
        if len(pro) > 0:
            #get rid of the part that is not part of the pronunciaiton
            defis = frag.replace(f"{pro}]", "")
            #print("pronunciation:", pro, ",definitions:", defis)
            defis = defis.split(";")#get separate defintions
            for d in range(0,len(defis)):
                defi =  defis[d].strip() #get a certain definition, with leading/ training whitespace remove
                newTrad = False #store if a new trad. character is found
                trad2 = getbetween(defi,"(F",")") #modify the fragment string
                if len(trad2) != 0: #if function is not empty, get to be the same as the simplified character
                    trad = trad2
                    newTrad = True
                
                print("current def:", defi)
                chinese_pattern  = re.compile(r'[\u4E00-\u9FFF]+') #regex to get list of matching characters 
                match = chinese_pattern.findall(defi)
                han = getall_betweenchars(match, defi)   #helper to get all chars between the list of matching characters 
                print("found subcharacter:"+han+"|")
                if len(han)>1 and newTrad==False and i!=0 and "=" not in defi: #if the def definition contains multiple chinese characters, we do extra processing to extract the characters, and if this is not a new trad 
                    charind = defi.find(han) #find last index
                    char = han.strip() # the character for the word 
                    defi = defi[charind+len(han):] #modify the definition to be the remaining part
                    new1 = [cat, char, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, char, pro, defi, "t",alt] #for now
                    ret.append(new1)
                    ret.append(new2)
                    #in this case, we append as so 
                    print("ran if"+str(new2))
                else: #assume that there is only one character to the definition
                    new1 = [cat, simp, pro, defi, "s",alt] #"cat", "word/charcter", "pronunciation", "definition", "code","alt"
                    new2 = [cat, trad, pro, defi, "t",alt]
                    ret.append(new1)
                    ret.append(new2)
                    print("ran else"+str(new2))
                #print("character:"+char+", definition:"+defi)
                i+= 1

    return ret 

In [3]:
link = "./main_char_set-converted.html"

with open(link, 'r') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')
tables = soup.find_all("table")  
grand_list = [] #list used to create the dataframe in the end
#count the number of tables 
index = 0;
for table in tables:
    rows = table.find_all("tr") #extract all tables
    for row in rows: 
        # Extract all cells (td or th)
        cols = row.find_all(["td"]) #print the number of cols (exclude the header) 
        #valid character columns have 3 columns 
        if len(cols) == 3:
            #get the number column 
            col1 = cols[0].get_text()
            #get the char column 
            col2 = cols[1].get_text()
            #get description column, this is complicated and is where we split the characters from the words, using a helper function
            col3 = cols[2].get_text()
            #call our key helper function that retruns a list to be appended to the grand_list 
            res = get_wordList(col2, col3, cat=index)
            grand_list.append(res)
            index += 1
print("total number of characters listed:", index)

current simp. :Caractère, current trad. :Caractère
current pronunciation:Prononciation et explications
current def: Prononciation et explications
found subcharacter:|
ran else[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 't', '']
current simp. :的, current trad. :的
current pronunciation:
current pronunciation:de
current def: <grammatical particle marking genitive as well as simple and composed adjectives>
found subcharacter:|
ran else[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
current def: 我的 wǒde my
found subcharacter:我的|
ran if[1, '我的', 'de', ' wǒde my', 't', '']
current def: 高的 gāode high, tall
found subcharacter:高的|
ran if[1, '高的', 'de', ' gāode high, tall', 't', '']
current def: 是的 shìde that's it, that's right
found subcharacter:是的|
ran if[1, '是的', 'de', " shìde that's it, that's right", 't', '']
current def: 是...的 shì...de one who...
found subcharacter:是...的 |
ran if[1, '是...的', 

In [4]:
print(len(grand_list))
grand_list

2431


[[[0,
   'Caractère',
   'Prononciation et explications',
   'Prononciation et explications',
   's',
   ''],
  [0,
   'Caractère',
   'Prononciation et explications',
   'Prononciation et explications',
   't',
   '']],
 [[1,
   '的',
   'de',
   '<grammatical particle marking genitive as well as simple and composed adjectives>',
   's',
   ''],
  [1,
   '的',
   'de',
   '<grammatical particle marking genitive as well as simple and composed adjectives>',
   't',
   ''],
  [1, '我的', 'de', ' wǒde my', 's', ''],
  [1, '我的', 'de', ' wǒde my', 't', ''],
  [1, '高的', 'de', ' gāode high, tall', 's', ''],
  [1, '高的', 'de', ' gāode high, tall', 't', ''],
  [1, '是的', 'de', " shìde that's it, that's right", 's', ''],
  [1, '是的', 'de', " shìde that's it, that's right", 't', ''],
  [1, '是...的', 'de', 'shì...de one who...', 's', ''],
  [1, '是...的', 'de', 'shì...de one who...', 't', ''],
  [1,
   '他是说汉语的',
   'de',
   '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.',
   's',
   ''],
  [1,
   '他是

As expected, the list has 2431 subcomponents, but to effectively create the dataframe we need to remove a layer. 

In [5]:
master_list = []
for l1 in grand_list:
    for l2 in l1:
        print(l2)
        master_list.append(l2)

[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 's', '']
[0, 'Caractère', 'Prononciation et explications', 'Prononciation et explications', 't', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 's', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
[1, '我的', 'de', ' wǒde my', 's', '']
[1, '我的', 'de', ' wǒde my', 't', '']
[1, '高的', 'de', ' gāode high, tall', 's', '']
[1, '高的', 'de', ' gāode high, tall', 't', '']
[1, '是的', 'de', " shìde that's it, that's right", 's', '']
[1, '是的', 'de', " shìde that's it, that's right", 't', '']
[1, '是...的', 'de', 'shì...de one who...', 's', '']
[1, '是...的', 'de', 'shì...de one who...', 't', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 's', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 't', '']
[1, '目的', 'dì', ' mùdì goal', 's', '']
[1,

Saving to a new csv file. First, remove the last two and first two entries from the, its clean its garbage

In [6]:
master_list = master_list[2:len(master_list)-2]
for l in master_list:
    print(l)

[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 's', '']
[1, '的', 'de', '<grammatical particle marking genitive as well as simple and composed adjectives>', 't', '']
[1, '我的', 'de', ' wǒde my', 's', '']
[1, '我的', 'de', ' wǒde my', 't', '']
[1, '高的', 'de', ' gāode high, tall', 's', '']
[1, '高的', 'de', ' gāode high, tall', 't', '']
[1, '是的', 'de', " shìde that's it, that's right", 's', '']
[1, '是的', 'de', " shìde that's it, that's right", 't', '']
[1, '是...的', 'de', 'shì...de one who...', 's', '']
[1, '是...的', 'de', 'shì...de one who...', 't', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 's', '']
[1, '他是说汉语的', 'de', '. Tā shì shuō Hànyǔde. He is one who speaks Chinese.', 't', '']
[1, '目的', 'dì', ' mùdì goal', 's', '']
[1, '目的', 'dì', ' mùdì goal', 't', '']
[1, '的', 'dí', 'true, real', 's', '']
[1, '的', 'dí', 'true, real', 't', '']
[1, '的确', 'dí', ' díquè certainly', 's', '']
[1, '的确', 'dí', ' díquè cer

We now save to a dataframe 

In [7]:
data =  pd.DataFrame(master_list, columns=["cat", "word/charcter", "pronunciation", "definition", "code","alt"])
data.to_csv("main_list.csv", index=False)


Scratch: testing regex

In [8]:
text = "(F 幹) do{天干 tiāngān the Ten Heavenly Stems: 甲乙丙丁戊己庚辛壬癸 jiǎ-yǐ-bǐng-dīng-wù-jǐ-gēng-xīn-rén-guǐ (these characters aresometimes used as ordinal numbers}{Compare with 千 qiān thousand and 于 yú at}"
trad2 = getbetween(text,"(F",")") #modify the fragment string
print(trad2)

幹
